In [3]:
import sys
import os
sys.path.append('../') # root directory

In [15]:
import read.JsonFileReader as json_reader
from write.JsonFileWriter import write
from constants import constant
import pandas as pd
import numpy as np
import read.XlsReader as reader

In [13]:
EMPTY = chr(8212)
US_STOCKS = '../output/USIDX_2020-08-23114120744101.json'

In [19]:
all_stat_names = [stat['name'] for key in ('financeStats', 'keyStats') for stat in stocks[0][key]]
all_stat_names[:10]

['Revenue',
 'Gross Margin %',
 'Operating Income',
 'Operating Margin %',
 'Net Income',
 'Earnings Per Share',
 'Dividends',
 'Payout Ratio % *',
 'Shares',
 'Book Value Per Share *']

In [112]:
# TODO deal with statNames order

In [113]:
def getrow(stock, stat_names, periods='all', keys=('financeStats', 'keyStats')):
    stats = [stat for key in keys for stat in stock[key] if stat['name'] in stat_names]
    stat_values = [v['value'] for stat in stats for v in stat['values'] if periods == 'all' or v['period'] in periods]
    return [stock['symbol']] + stat_values

In [172]:
def extract_stat(stocks, stat_names, periods='all', keys=('financeStats', 'keyStats')):
    stats = [getrow(stock, stat_names, periods, keys) for stock in stocks]
    colheaders = pd.MultiIndex.from_product([stat_names, periods]).insert(0, 'Symbol')
    stats_df = pd.DataFrame(stats, columns=colheaders)
    stats_df = stats_df.replace(EMPTY, np.NaN)
    stats_df.loc[:, stat_names] = stats_df.loc[:, stat_names].astype('float32')
    return stats_df

In [173]:
stat_names = ['Earnings Per Share']
keys = ('financeStats',)
periods = ['2015-12', '2016-12', '2017-12', '2018-12', '2019-12', 'TTM']

In [174]:
stats_df = extract_stat(stocks, stat_names, periods, keys=keys)

/Users/vidurpuliani/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: PerformanceWarning: indexing past lexsort depth may impact performance.
  interactivity=interactivity, compiler=compiler, result=result)


In [216]:
eido = pd.merge(eido_df, stats_df, on='Symbol', how='left')
eido['Weight'] * eido[('Earnings Per Share', 'TTM')]

/Users/vidurpuliani/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/vidurpuliani/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


0     21640.300000
1      3092.260567
2      1699.403268
3      3312.000000
4      3452.248674
          ...     
69             NaN
70       -1.266800
71             NaN
72             NaN
73             NaN
Length: 74, dtype: float64

In [215]:
eido

,Symbol,Name,Sector,Asset,MarketCap,Weight,"(Earnings Per Share, 2015-12)","(Earnings Per Share, 2016-12)","(Earnings Per Share, 2017-12)","(Earnings Per Share, 2018-12)","(Earnings Per Share, 2019-12)","(Earnings Per Share, TTM)"
0,BBCA,BANK CENTRAL ASIA,Financials,Equity,"59,178,550.05",19.10,731.000000,836.000000,945.000000,1049.000000,1159.000000,1133.000000
1,BBRI,BANK RAKYAT INDONESIA (PERSERO),Financials,Equity,"34,187,697.99",11.03,206.089996,214.070007,236.929993,264.369995,280.959991,280.350006
2,TLKM,TELEKOMUNIKASI INDONESIA,Communication,Equity,"27,488,397.69",8.87,157.380005,195.990005,223.300003,179.710007,192.490005,191.589996
3,ASII,ASTRA INTERNATIONAL,Consumer Discretionary,Equity,"17,851,588.63",5.76,357.000000,374.000000,466.000000,535.000000,536.000000,575.000000
4,BMRI,BANK MANDIRI (PERSERO),Financials,Equity,"17,712,663.82",5.72,435.750000,295.850006,442.279999,536.039978,588.900024,603.539978
...,...,...,...,...,...,...,...,...,...,...,...,...
69,BLTA,PT BERLIAN LAJU TANKER TBK,Industrials,Equity,"70,841.33",0.02,0.010000,NaN,NaN,NaN,NaN,NaN
70,POOL,POOL ADVISTA INDONESIA,Financials,Equity,"22,910.27",0.01,11.550000,19.600000,103.440002,-27.770000,-180.130005,-126.680000
71,SIAP,SEKAWAN INTIPRATAMA,Materials,Equity,20.69,0.00,-1.070000,-0.920000,-0.450000,-0.480000,-0.480000,NaN
72,SUGI,SUGIH ENERGY,Energy,Equity,27,0.00,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
'http://financials.morningstar.com/ajax/ReportProcess4HtmlAjax.html?&t=XIDX:BBCA&region=idn&culture=en-US&version=SAL&cur=&reportType=is&period=12&dataType=R&order=asc&columnYear=10&curYearPart=1st5year&rounding=3&view=decimal&r=401953&callback=jsonp1597682400046&_=1597682400090'

'http://financials.morningstar.com/ajax/ReportProcess4HtmlAjax.html?&t=XIDX:BBCA&region=idn&culture=en-US&version=SAL&cur=&reportType=is&period=12&dataType=R&order=asc&columnYear=10&curYearPart=1st5year&rounding=3&view=decimal&r=401953&callback=jsonp1597682400046&_=1597682400090'

In [46]:
'http://financials.morningstar.com/ajax/ReportProcess4HtmlAjax.html?&t=XNSE:PCJEWELLER&region=ind&culture=en-US&version=SAL&cur=&reportType=is&period=12&dataType=R&order=asc&columnYear=10&curYearPart=1st5year&rounding=3&view=decimal&r=515122&callback=jsonp1597682464958&_=1597682465025'

'http://financials.morningstar.com/ajax/ReportProcess4HtmlAjax.html?&t=XNSE:PCJEWELLER&region=ind&culture=en-US&version=SAL&cur=&reportType=is&period=12&dataType=R&order=asc&columnYear=10&curYearPart=1st5year&rounding=3&view=decimal&r=515122&callback=jsonp1597682464958&_=1597682465025'

In [47]:
'http://financials.morningstar.com/ajax/ReportProcess4HtmlAjax.html?&t=XNSE:PCJEWELLER&region=ind&culture=en-US&version=SAL&cur=&reportType=bs&period=12&dataType=R&order=asc&columnYear=10&curYearPart=1st5year&rounding=3&view=decimal&r=617433&callback=jsonp1597682521236&_=1597682521326'

'http://financials.morningstar.com/ajax/ReportProcess4HtmlAjax.html?&t=XNSE:PCJEWELLER&region=ind&culture=en-US&version=SAL&cur=&reportType=bs&period=12&dataType=R&order=asc&columnYear=10&curYearPart=1st5year&rounding=3&view=decimal&r=617433&callback=jsonp1597682521236&_=1597682521326'

ADR: false
columnYear: "10" or "5"
culture: "en-US"
dataType: "A"(As Reported), "R"(Restated)
order: "asc" or "desc"
period: "12" or "3"
reportType: "bs" or "is" or "cf"
rounding: "4", 1(original), 2(thousands), 3(millions), 4(billions)
userType: "S"
view: "raw" or "percentage" or "decimal"